In [1]:
import sqlite3
conn = sqlite3.connect('lab4.db')

c = conn.cursor()

In [2]:
# c.execute('''DROP TABLE gene;''')
# c.execute('''DROP TABLE enzyme;''')
# c.execute('''DROP TABLE en_pa;''')
# c.execute('''DROP TABLE pathway;''')

In [3]:
c.execute('''CREATE TABLE gene
             (id INT, ncbi_id TEXT, name_desc TEXT, organism TEXT, chr TEXT, start INT, end INT, eid INT, sequence TEXT )''')

c.execute('''CREATE TABLE enzyme
            (id INT, name TEXT, function TEXT, ecnum TEXT )''')

c.execute('''CREATE TABLE en_pa (eid INT, pid INT, priority INT )''')

c.execute('''CREATE TABLE pathway
             (id INT, name TEXT, description TEXT )''')

In [4]:
# Add pathway

pa_file = open("lab4_pathway_data")

name = ''
description = ''

iterator = 1

for i, line in enumerate(pa_file):

    line = line.replace('\n', '')
    
    if '#' in line:
        continue
        
    if i%3 is 0:
        name = line
    elif i%3 is 1:
        description = line
    elif i%3 is 2:
        c.execute("INSERT INTO pathway (id, name, description) VALUES ("+ str(iterator) + ", '" + name + "', '" + description + "' )" )        
        iterator+=1

conn.commit()


In [5]:
# Add enzyme

enz_file = open("lab4_enzyme_data")

name = ''
pathorder = []
function = ''
ECnum = ''

iterator = 1


for i, line in enumerate(enz_file):

    line = line.replace('\n', '')
    
    if '#' in line:
        continue
    if i%5 is 0:
        name = line
    elif i%5 is 1:
        pathorder = line.split(',')
        for i, item in enumerate(pathorder):
            pathorder[i] = pathorder[i].split('|')
            pathorder[i][1] = int(pathorder[i][1])
    elif i%5 is 2:
        function = line
    elif i%5 is 3:
        ECnum = line
    elif i%5 is 4:
        c.execute("INSERT INTO enzyme (id, name, function, ecnum) VALUES ("+ str(iterator) + ", '" + name + "', '" + function +"', '"+ ECnum + "' )" )
        
        ### TODO ###
        for rep in range(len(pathorder)):
            c.execute("INSERT INTO en_pa (eid, pid, priority) VALUES ("+ str(iterator) + "," + "(select id from pathway where name = "+"'"+pathorder[rep][0]+"')"+" ,"+str(pathorder[rep][1])+")")
            
        iterator+=1
        
conn.commit()

In [6]:
# #Search for Genes: Drosophila_melanogaster : Drosophila_melanogaster_sequen.fasta
# from Bio import Entrez
# from Bio import SeqIO
# from Bio.SeqRecord import SeqRecord

# #Enzyme List:
# #               Triosephosphate isomerase   
# #                 Phosphoglycerate kinase   
# #         Phosphoenolpyruvate carboxylase   
# #                         Pyruvate kinase   
# #       Glucose-6-phosphate dehydrogenase   
# #                        Gluconolactonase   
# #        6-phosphogluconate dehydrogenase   
# #                           Transaldolase   
# #                Succinate dehydrogenase   
# #                               Fumarase   
# #                  Malate dehydrogenase   
# #                      Citrate synthase

# # Possibble db




# Entrez.email = 'mike.ferna512@berkeley.edu'
# handle = Entrez.esearch(db='nucleotide',
#                        term='E coli Triosephosphate isomerase',
#                        sort='relevance',
#                        idtype='acc')

# for i in Entrez.read(handle)['IdList']:
#     handle = Entrez.efetch(db='nucleotide',id = i, rettype='fasta', retmode='text', retmax = 1)
#     ids = i;
#     break
# # make sure to view sequence before adding to database to be sure it is the correct sequence
# handle = Entrez.efetch(db="nucleotide", id= ids, rettype="fasta", retmode="text")
# record = handle.read()
# print(record)





In [7]:
# #----------------------#
# # Add sequence
    
# #record1 = open('Drosophila_melanogaster_sequen.fasta', 'a+')
# record1 = open('E_Coli_sequen.fasta', 'a+')
# #record1 = open('Homo_sapiens_sequen.fasta', 'a+')

# handle = Entrez.efetch(db="nucleotide", id= ids, rettype="fasta", retmode="text")
# record = handle.read()
# print(record)
# record1.write(record) 


In [8]:
# Add Gene

from Bio import SeqIO
import pandas as pd


gene_file = ['Drosophila_melanogaster_sequen.fasta', 'E_Coli_sequen.fasta', 'Homo_sapiens_sequen.fasta']
org_list = ['Drosophila melanogaster', 'Escherichia coli', 'Homo sapiens']
enz_list =    ['Triose phosphate isomerase' ,
                'Phosphoglycerate kinase'  ,
        'Phosphoenolpyruvate carboxylase'  ,
                        'Pyruvate kinase'  ,
      'Glucose-6-phosphate dehydrogenase'  ,
                       'Gluconolactonase'  ,
       '6-phosphogluconate dehydrogenase'  ,
                          'Transaldolase' ,
               'Succinate dehydrogenase' ,
                              'Fumarase', 
                 'Malate dehydrogenase',
                     'Citrate synthase' ]

chrRule = ['X', 'Y', 'R', 'L', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
# c.execute('''CREATE TABLE gene
#              (id INT, ncbi_id TEXT, name_desc TEXT, organism TEXT, chr TEXT, start INT, end INT, eid INT, sequence TEXT )''')

iterator = 1
for file in gene_file:
    for gene in SeqIO.parse(file, 'fasta'):
        
    # analysing information from FASTA
        
    # 1. id : iterating gene
        order_id = iterator
        
    # 2. ncbi_id : ncbi gene id
        ncbi_id = gene.id
        
    # 3. name_desc : name, description from fasta
        name_desc = gene.description[len(gene.id)+1: : ]
        
    # 4. organism
        organism = ''
        for org in org_list:
            if org in gene.description:
                organism = org
                break
                
    # 5. chromosome : (blank if not stated)
        chromosome = ''
        try:
            scan = str(gene.description)
            idx = scan.index('chromosome')
        
            while( scan[idx+11] in chrRule ):
                chromosome+=scan[idx+11]
                idx+=1
        except:
            pass
        
    # 6,7. st, en : starting / position in the gene
        st, en = '', ''
        
        try:
            scan = str(gene.name)
            idx = scan.index(':c')
            
            while( scan[idx+2] in chrRule ):
                st+=scan[idx+2]
                idx+=1
            idx+=1
            while( scan[idx+2] in chrRule ):
                en+=scan[idx+2]
                idx+=1
                if idx+2 is len(scan):
                    break
            st = int(st)
            en = int(en)
            
        except:
            st=0
            en=0
            pass


        
    # 8. eid : match enzyme id from the db
        eid = 0
        
        scan = str(gene.description)
        
        #print(scan)
        for enz in enz_list:
            if scan.lower().replace(' ', '').find(enz.lower().replace(' ','')) is not -1:
                input_str = enz[0].upper() + enz[1:].lower()
                output = str(pd.read_sql_query("SELECT id FROM enzyme WHERE name=="+"'"+str(input_str)+"'"+ ";", conn ))

                tmp = []
                idx = -1
                while( output[idx] is not ' '):
                    tmp.append(output[idx])
                    idx-=1

                tmp.reverse()
                eid = int("".join(tmp))
                break
                
    #9. seq : nt-sequence
        
        seq = str(gene.seq)
        
    # now, lets put to the db!
    
        c.execute("""INSERT INTO gene
             (id , ncbi_id , name_desc , organism , chr , start, end, eid, sequence )
             VALUES ("""
                 + str(order_id) + ', '
                 + "'" + ncbi_id + "', "
                 + "'" + name_desc + "', "
                 + "'" + organism + "', "
                 + "'" + chromosome + "', "
                 + str(st) + ', '
                 + str(en) + ', '
                 + str(eid)+ ', '
                 + "'" + seq + "'"
             
             +')'
        )
        

        iterator+=1
        
    conn.commit()

        
    

In [9]:
# print table

import pandas as pd

print (pd.read_sql_query("SELECT * FROM gene;",conn))
print (pd.read_sql_query("SELECT * FROM enzyme;", conn))
print (pd.read_sql_query("SELECT * FROM pathway;", conn))
print (pd.read_sql_query("SELECT * FROM en_pa;", conn))

    id                            ncbi_id  \
0    1     NT_033777.3:c30134599-30132981   
1    2       NT_033779.5:c2753068-2746880   
2    3         NZ_KB233235.1:c23004-20365   
3    4                        NM_079724.3   
4    5                         AH002543.2   
5    6      NC_004354.4:12012377-12017375   
6    7                           M80598.1   
7    8                        NM_079111.3   
8    9                        NM_057862.5   
9   10                     NM_001298874.1   
10  11       NC_000913.3:c4111507-4110740   
11  12       NC_000913.3:c3072622-3071459   
12  13       NC_000913.3:c4153098-4150447   
13  14        NC_000913.3:1755698-1757110   
14  15       NC_011751.1:c2171506-2170031   
15  16       NC_000913.3:c2101268-2099862   
16  17        NC_000913.3:2578666-2579616   
17  18          NC_000913.3:755907-757673   
18  19       NC_000913.3:c1686588-1685185   
19  20       NC_000913.3:c3384268-3383330   
20  21         NC_000913.3:c754468-753185   
21  22    

In [10]:
# view by enzyme priority in each pathway SORTED BY ORDER!!

print(pd.read_sql_query("SELECT enzyme.id, enzyme.name FROM enzyme INNER JOIN en_pa on enzyme.id = en_pa.eid WHERE en_pa.pid = 2 and en_pa.priority>0 ORDER BY en_pa.priority ASC ;", conn))
# sort enzymes by iwhere pid=1 and have priority information(priority>0)


   id                               name
0   5  Glucose-6-phosphate dehydrogenase
1   6                   Gluconolactonase
2   7   6-phosphogluconate dehydrogenase
3   8                      Transaldolase
